# Day 14

## Part One

### Thoughts and observations

Example recipe:

```
10 ORE => 10 A
1 ORE => 1 B
7 A, 1 B => 1 C
7 A, 1 C => 1 D
7 A, 1 D => 1 E
7 A, 1 E => 1 FUEL
```

Let's try making map `Target => [(count_to_be, needed_sources), ...]`:

In [25]:
use std::{
    collections::{
        HashMap as Map,
        HashSet as Set,
    },
    convert::TryFrom,
    str::FromStr,
};

#[derive(Debug)]
struct Material<'a> {
    count: usize,
    name: &'a str,
}

#[derive(Debug)]
struct Reaction<'a> {
    sources: Vec<Material<'a>>,
    target: Material<'a>,
}

#[derive(Debug)]
struct InvalidFormat;

impl<'a> TryFrom<&'a str> for Material<'a> {
    type Error = InvalidFormat;
    
    fn try_from(s: &'a str) -> Result<Self, Self::Error> {
        let mut splitted = s.split(' ');
        let count = splitted
            .next()
            .ok_or(InvalidFormat)?
            .parse()
            .map_err(|_| InvalidFormat)?;
        
        let name = splitted
            .next()
            .ok_or(InvalidFormat)?;
        
        Ok(Self { count, name })
    }
}

impl<'a> TryFrom<&'a str> for Reaction<'a> {
    type Error = InvalidFormat;
    
    fn try_from(s: &'a str) -> Result<Self, Self::Error> {
        let mut splitted = s.split(" => ");

        let sources = splitted.next().unwrap();
        let target = splitted.next().unwrap();

        let sources = sources
            .split(", ")
            .map(Material::try_from)
            .collect::<Result<Vec<Material>, _>>()?;

        let target: Material = Material::try_from(target)?;
        
        Ok(Self { sources, target })
    }
}

fn parse_input<'a>(recipe_str: &'a str) -> Map<&'a str, Vec<Reaction<'a>>> {
    let mut map = Map::new();

    for line in recipe_str.split('\n') {
        let reaction = Reaction::try_from(line).unwrap();

        map
            .entry(reaction.target.name)
            .or_insert_with(|| Vec::new())
            .push(reaction);
    }
    
    map
}

let recipe = "10 ORE => 10 A
1 ORE => 1 B
7 A, 1 B => 1 C
7 A, 1 C => 1 D
7 A, 1 D => 1 E
7 A, 1 E => 1 FUEL";

println!("{:#?}", parse_input(recipe));

Error: the name `Map` is defined multiple times

Error: the name `TryFrom` is defined multiple times

Error: the name `FromStr` is defined multiple times

Error: no field `target` on type `&std::vec::Vec<Reaction<'_>>`

Error: no field `target` on type `&std::vec::Vec<Reaction<'_>>`

Error: no field `sources` on type `&std::vec::Vec<Reaction<'_>>`

Let's check if there is any ambiguous recipies (eg. `1 A, 1 B => 1 C; 1 D, 1 E => 2 C`)

In [15]:
let recipe = include_str!("/home/utterstep/my/advent-2019/day-14/input.txt").trim();

// this is not valid code currently
for (material, recipes) in parse_input(recipe).iter().filter(|(_material, recipes)| recipes.len() > 1) {
    println!("{} can be made from these ingredients: {:#?}", material, recipes);
}

()

Well, that simplifies the scheme a little.

Let's try to do some backtracking — get the needed ingredients for the `FUEL`, mark them as needed (taking into account needed quantity), for every one of them let's get their ingredients and repeat this process until we only have `ORE` as requirements.

It wouldn't be an optimal solution (in terms of used `ORE` and other materials), but at least it would be some starting point.

In [ ]:
// rewrite parser, removing ambigous recipes possibility

fn parse_input<'a>(recipe_str: &'a str) -> Map<&'a str, Reaction<'a>> {
    let mut map = Map::new();

    for line in recipe_str.split('\n') {
        let reaction = Reaction::try_from(line).unwrap();

        let prev = map.insert(reaction.target.name, reaction);
        
        assert!(prev.is_none(), "ambigous recipe found")
    }
    
    map
}

Let's try brute force approach.

In [35]:
// let's start with simple example

let recipe = "10 ORE => 10 A
1 ORE => 1 B
7 A, 1 B => 1 C
7 A, 1 C => 1 D
7 A, 1 D => 1 E
7 A, 1 E => 1 FUEL";

fn solve_brute(recipe_str: &str) -> usize {
    let recipes_data = parse_input(recipe_str);

    const FUEL: &str = "FUEL";
    const ORE: &str = "ORE";

    let mut to_resolve = vec![(1, FUEL)];
    let mut ore_requirements = 0;

    while !to_resolve.is_empty() {
        let (count, material) = to_resolve.pop().unwrap();

        let reaction = &recipes_data[material];

        let coeff = count / reaction.target.count + if count % reaction.target.count > 0 { 1 } else { 0 };

        for req in reaction.sources.iter() {
            if req.name == ORE {
                ore_requirements += req.count * coeff;
            } else {
                to_resolve.push((req.count * coeff, req.name));
            }
        }
    }

    ore_requirements
}

solve_brute(recipe)

41

Yup :(

The correct answer is `31`, but my algorithm yields `41`, because as soon as it sees `7A` it makes an "order" for 10A (minimum increment), while correct strategy would be to merge orders for `A`, because they do not rely on each other, and execute them as `7A + 7A + 7A + 7A = 28A` -> `3 * (10 ORE => 10 A)`.

In [66]:
use std::collections::HashSet as Set;

let recipe = "10 ORE => 10 A
1 ORE => 1 B
7 A, 1 B => 1 C
7 A, 1 C => 1 D
7 A, 1 D => 1 E
7 A, 1 E => 1 FUEL";
// let recipe = include_str!("/home/utterstep/my/advent-2019/day-14/input.txt").trim();

const FUEL: &str = "FUEL";
const ORE: &str = "ORE";

fn topological_sort<'a>(recipe: &Map<&'a str, Reaction<'a>>, start: &'a str) -> Vec<&'a str> {
    let mut order = vec![];
    let mut visited = Set::new();
    
    fn dfs<'a>(
        node: &'a str,
        order: &mut Vec<&'a str>,
        visited: &mut Set<&'a str>, 
        recipe: &Map<&'a str, Reaction<'a>>
    ) {
        if visited.insert(node) {
            for source in &recipe[node].sources {
                if source.name != ORE {
                    dfs(
                        source.name,
                        order,
                        visited,
                        recipe,
                    );
                }
            }
            
            order.push(node);
        }
    }
    
    dfs(
        start,
        &mut order,
        &mut visited,
        recipe,
    );
    
    order.reverse();
    order.push(ORE);
    
    order
}

struct SolutionPrecalc<'a> {
    recipe_data: Map<&'a str, Reaction<'a>>,
    material_order: Vec<&'a str>,
}

fn get_precalc(recipe_str: &str) -> SolutionPrecalc {
    let recipe_data = parse_input(recipe_str);
    
    let material_order = topological_sort(&recipe_data, FUEL);
    
    SolutionPrecalc {
        recipe_data,
        material_order,
    }
}

fn compute_ore_requirements(precalc: &SolutionPrecalc, fuel_required: usize) -> usize {
    let mut requirements = Map::new();
    requirements.insert(FUEL, fuel_required);

    for material in &precalc.material_order {
        if let Some(reaction) = precalc.recipe_data.get(material) {
            let count = requirements[material];
            let coeff = count / reaction.target.count + if count % reaction.target.count > 0 { 1 } else { 0 };

            for source in &reaction.sources {
                (*requirements.entry(source.name).or_insert(0)) += source.count * coeff;
            }
        }
    }
    
    requirements[ORE]
}

fn solve_easy(recipe_str: &str) -> usize {
    let precalc = get_precalc(recipe_str);

    compute_ore_requirements(&precalc, 1)
}

solve_easy(recipe)

31

Hooray! Let's check other examples

In [71]:
const RECIPES: &[(&str, usize)] = &[
    (
        "9 ORE => 2 A
8 ORE => 3 B
7 ORE => 5 C
3 A, 4 B => 1 AB
5 B, 7 C => 1 BC
4 C, 1 A => 1 CA
2 AB, 3 BC, 4 CA => 1 FUEL",
        165,
    ),
    (
        "157 ORE => 5 NZVS
165 ORE => 6 DCFZ
44 XJWVT, 5 KHKGT, 1 QDVJ, 29 NZVS, 9 GPVTF, 48 HKGWZ => 1 FUEL
12 HKGWZ, 1 GPVTF, 8 PSHF => 9 QDVJ
179 ORE => 7 PSHF
177 ORE => 5 HKGWZ
7 DCFZ, 7 PSHF => 2 XJWVT
165 ORE => 2 GPVTF
3 DCFZ, 7 NZVS, 5 HKGWZ, 10 PSHF => 8 KHKGT",
        13312,
    ),
    (
        "2 VPVL, 7 FWMGM, 2 CXFTF, 11 MNCFX => 1 STKFG
17 NVRVD, 3 JNWZP => 8 VPVL
53 STKFG, 6 MNCFX, 46 VJHF, 81 HVMC, 68 CXFTF, 25 GNMV => 1 FUEL
22 VJHF, 37 MNCFX => 5 FWMGM
139 ORE => 4 NVRVD
144 ORE => 7 JNWZP
5 MNCFX, 7 RFSQX, 2 FWMGM, 2 VPVL, 19 CXFTF => 3 HVMC
5 VJHF, 7 MNCFX, 9 VPVL, 37 CXFTF => 6 GNMV
145 ORE => 6 MNCFX
1 NVRVD => 8 CXFTF
1 VJHF, 6 MNCFX => 4 RFSQX
176 ORE => 6 VJHF",
        180697,
    ),
    (
        "171 ORE => 8 CNZTR
7 ZLQW, 3 BMBT, 9 XCVML, 26 XMNCP, 1 WPTQ, 2 MZWV, 1 RJRHP => 4 PLWSL
114 ORE => 4 BHXH
14 VRPVC => 6 BMBT
6 BHXH, 18 KTJDG, 12 WPTQ, 7 PLWSL, 31 FHTLT, 37 ZDVW => 1 FUEL
6 WPTQ, 2 BMBT, 8 ZLQW, 18 KTJDG, 1 XMNCP, 6 MZWV, 1 RJRHP => 6 FHTLT
15 XDBXC, 2 LTCX, 1 VRPVC => 6 ZLQW
13 WPTQ, 10 LTCX, 3 RJRHP, 14 XMNCP, 2 MZWV, 1 ZLQW => 1 ZDVW
5 BMBT => 4 WPTQ
189 ORE => 9 KTJDG
1 MZWV, 17 XDBXC, 3 XCVML => 2 XMNCP
12 VRPVC, 27 CNZTR => 2 XDBXC
15 KTJDG, 12 BHXH => 5 XCVML
3 BHXH, 2 VRPVC => 7 MZWV
121 ORE => 7 VRPVC
7 XCVML => 6 RJRHP
5 BHXH, 4 VRPVC => 5 LTCX",
        2210736,
    )
];

for (recipe, required) in recipes {
    assert_eq!(solve_easy(recipe), *required);
}

println!("Everything is OK!");

Everything is OK!


In [72]:
// let's solve the task!

let recipe = include_str!("/home/utterstep/my/advent-2019/day-14/input.txt").trim();

solve_easy(recipe)

628586

### Part Two

After collecting `ORE` for a while, you check your cargo hold: 1 trillion (1000000000000) units of `ORE`.

With that much ore, given the examples above:

    The 13312 ORE-per-FUEL example could produce 82892753 FUEL.
    The 180697 ORE-per-FUEL example could produce 5586022 FUEL.
    The 2210736 ORE-per-FUEL example could produce 460664 FUEL.

Given 1 trillion `ORE`, what is the maximum amount of `FUEL` you can produce?

In [78]:
const ORE_CARGO: usize = 1_000_000_000_000;

fn solve_hard(recipe_str: &str) -> usize {
    let mut hi = ORE_CARGO;
    let mut lo = 1;
    
    let precalc = get_precalc(recipe_str);
    
    loop {
        let mid = (hi + lo) / 2;
        let required_ore = compute_ore_requirements(&precalc, mid);
        
        if required_ore > ORE_CARGO {
            hi = mid - 1;
        } else if required_ore < ORE_CARGO {
            lo = mid;
        } else {
            return mid;
        }
        
        if hi - lo <= 1 {
            return lo;
        }
    }
}

solve_hard(recipe)

3209254

Ta-da!